In [ ]:
from miditok import REMI, TokenizerConfig
from symusic import Score
from tokenizers import Tokenizer, models, pre_tokenizers, trainers
from tokenizers.pre_tokenizers import Split

from tqdm import tqdm
import os
from tokenizers import decoders, processors
from tokenizers import Tokenizer, models, pre_tokenizers, trainers, normalizers
import os
from tokenizers.pre_tokenizers import CharDelimiterSplit
from tokenizers.processors import TemplateProcessing
from transformers import AutoTokenizer

# Load a pretrained tokenizer (BERT in this case)
w_tokenizer = AutoTokenizer.from_pretrained("/tmp2/b11902010/dmir_lab/mdlm/tokenizers/tokenizer_test")
TOKENIZER_PARAMS = {
    "pitch_range": (21, 109),
    "beat_res": {(0, 4): 8, (4, 12): 4},
    "num_velocities": 32,
    "special_tokens": ["[PAD]", "[BOS]", "[EOS]", "[MASK]"],  # Adjusted for Hugging Face compatibility
    "use_chords": True,
    "chord_maps": {
        "+" : (0, 4, 8), "/o7" : (0, 3, 6, 10), "7" : (0, 4, 7, 10),
        "M" : (0, 4, 7), "M7" : (0, 4, 7, 11), "m" : (0, 3, 7),
        "m7" : (0, 3, 7, 10), "o" : (0, 3, 6), "o7" : (0, 3, 6, 9),
        "sus2" : (0, 2, 7), "sus4" : (0, 5, 7)
    },
    "chord_tokens_with_root_note": True,
    "use_rests": False,
    "use_tempos": False,
    "use_time_signatures": False,
    "use_programs": False,
    "num_tempos": 32,  # Number of tempo bins
    "tempo_range": (50, 180),  # (min, max)
}

config = TokenizerConfig(**TOKENIZER_PARAMS)
tokenizer = REMI(config)


# Open the file in read mode

folder = 'uncond_good'
mod = 13
folder_path = f"/tmp2/b11902010/dmir_lab/mdlm/outputs/samples/testing/midi/"

# Create the directory if it doesn't exist
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

with open(f"/tmp2/b11902010/dmir_lab/mdlm/outputs/samples/testing/text", "a") as file:

    ignored_tokens = {"[BOS]", "[UNK]", "[PAD]", "[EOS]", "[MASK]"}  # Define tokens to skip
    for idx, line in enumerate(file):
        data = line.strip().split()  # Split the line into tokens
        filtered_data = [x for x in data if x not in ignored_tokens]
        token_ids = [tokenizer.vocab[x] for x in filtered_data]
        score = tokenizer.decode([token_ids])

        name = str(idx%mod).zfill(3)
        r = str(idx//mod).zfill(2)
        score.dump_midi(f'{folder_path}/{name}_{r}.mid')
